In [25]:

import sys
import tensorflow as tf
from tensorflow.keras import backend as K
tf.keras.backend.set_floatx('float32')

from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, UpSampling2D, Concatenate, BatchNormalization, Conv2DTranspose, Flatten, PReLU, Reshape, Dropout, AveragePooling2D, Add, Lambda, Layer
from tensorflow.keras.regularizers import l2
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
import numpy as np

In [26]:

class Autoencoder_ff(Model):
    def __init__(
        self,
        input_shape:int,
        latent_dim:int,
        layer_sizes:list = [700,300,100],
        regularisation:float = 1e-5,
        act_fct:str = 'tanh',
        drop_rate = 0.0,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)

        self.inn = Input(shape=(input_shape))


        # encoder
        self.encoder_layers = []
        for l in layer_sizes:
            self.encoder_layers.append(
                Dense(l,act_fct,use_bias=False,kernel_regularizer=l2(regularisation))
            )
            if drop_rate > 0.0:
                self.encoder_layers.append(Dropout(drop_rate))
        
        self.encoder_layers.append(
            Dense(latent_dim,act_fct,use_bias=False,kernel_regularizer=l2(regularisation))
        )


        # decoder
        self.decoder_layers = []
        for l in layer_sizes[::-1]:
            self.decoder_layers.append(
                Dense(l,act_fct,use_bias=False,kernel_regularizer=l2(regularisation))
            ) 
            if drop_rate > 0.0:
                self.decoder_layers.append(Dropout(drop_rate))
        # last layer
        self.decoder_layers.append(
            Dense(input_shape,'linear',use_bias=False,kernel_regularizer=l2(regularisation))
        )
        self.out = self.call(self.inn)

    def call(self, inputs, training=None):
        x = inputs
        for layer in self.encoder_layers:
            x = layer(x)
        for layer in self.decoder_layers:
            x = layer(x)
        return x
    

In [27]:
inn = np.random.rand(50,10)
inn.shape

(50, 10)

In [28]:
mdl = Autoencoder_ff(10,2,drop_rate=0.05)
mdl.compile(optimizer=Adam(learning_rate=0.001),loss='mse')
mdl.fit(inn,inn,epochs=50,verbose=0)
mdl.evaluate(inn)

2/2 [==============================] - 0s 4ms/step - loss: 0.0115


0.011486676521599293

In [30]:
l = len(mdl.encoder_layers)
mdl.summary()

Model: "autoencoder_ff_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_64 (Dense)            (None, 700)               7000      
                                                                 
 dropout_36 (Dropout)        (None, 700)               0         
                                                                 
 dense_65 (Dense)            (None, 300)               210000    
                                                                 
 dropout_37 (Dropout)        (None, 300)               0         
                                                                 
 dense_66 (Dense)            (None, 100)               30000     
                                                                 
 dropout_38 (Dropout)        (None, 100)               0         
                                                                 
 dense_67 (Dense)            (None, 2)            

In [32]:
inn = Input((2,))
out = inn
for layer in mdl.decoder_layers:
    out = layer(out)

decoder = Model(inn,out)


In [33]:
decoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 2)]               0         
                                                                 
 dense_68 (Dense)            (None, 100)               200       
                                                                 
 dropout_39 (Dropout)        (None, 100)               0         
                                                                 
 dense_69 (Dense)            (None, 300)               30000     
                                                                 
 dropout_40 (Dropout)        (None, 300)               0         
                                                                 
 dense_70 (Dense)            (None, 700)               210000    
                                                                 
 dropout_41 (Dropout)        (None, 700)               0     